In [13]:
import pandas as pd
import openpyxl
import re
import numpy as np
from pathlib import Path
import matplotlib.cm as cm
import matplotlib.pyplot as plt

Get indicator data

In [56]:
def process_bloomberg_sheet(df_raw, remove_header_rows):
    """
    Applies standard cleaning pipeline to a raw Bloomberg Excel export:
    1. Removes metadata header (first 5 rows).
    2. Standardizes Date column.
    3. Coerces numeric data.
    4. Resamples to Monthly End.
    """
    # A. Slice junk rows
    # CRITICAL NOTE: This assumes EVERY sheet has exactly 5 lines of metadata header.
    df = df_raw.iloc[remove_header_rows:].copy()
    
    # B. Standardize Date Column
    if 'Security' in df.columns:
        df = df.rename(columns={'Security': 'Date'})
    
    # Validation: Ensure we actually have a Date column
    if 'Date' not in df.columns:
        return None

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # C. Convert Numeric Data
    cols_to_convert = df.columns.drop('Date')
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    
    # D. Set Index and Resample
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    return df.resample('M').last()

# ---------------------------------------------------------
# 1. Get & Process 2-Year Yield Data
# ---------------------------------------------------------

# Reading directly (assuming file exists as per snippet, or add robust check if needed)
try:
    yields_raw = pd.read_excel('2_years_yields.xlsx', engine='openpyxl')
    yields_2yr = process_bloomberg_sheet(yields_raw, remove_header_rows=5)
except FileNotFoundError:
    print("Warning: '2_years_yields.xlsx' not found. Skipping Yields data.")
    yields_2yr = None

# Define Universe (Yields)
tickers = {
    'US': 'USGG2YR Index', 'UK': 'GUKG2 Index', 'Germany': 'GDBR2 Index',
    'Japan': 'GJGB2 Index', 'Switzerland': 'GSWISS02 Index', 'Australia': 'GACGB2 Index'
}

# ---------------------------------------------------------
# 2. Get & Process Price Data (Equity Markets)
# ---------------------------------------------------------

raw_sheets = pd.read_excel('Prices.xlsx', sheet_name=None)
processed_frames = []

for sheet_name, df_raw in raw_sheets.items():
    # 1. Clean the specific sheet
    clean_df = process_bloomberg_sheet(df_raw, remove_header_rows=0)
    
    if clean_df is not None:
        # 2. Generate identifier
        safe_name = re.sub(r'\W|^(?=\d)', '_', sheet_name)
        
        # 3. CRITICAL: Rename columns to include the asset name.
        # If you skip this, you will end up with 10 columns named "Price" 
        # and no way to know which asset is which.
        clean_df.columns = [f"{safe_name}_{col}" for col in clean_df.columns]
        
        # 4. Set Date as Index (Required for joining)
        if "Date" in clean_df.columns:
            clean_df = clean_df.set_index("Date")
        
        # Ensure the index is actual Datetime objects (avoids mismatch errors)
        clean_df.index = pd.to_datetime(clean_df.index)
        
        processed_frames.append(clean_df)

# --- JOIN OPERATION ---
if processed_frames:
    # axis=1 joins columns (aligning by Date index)
    # sort=True sorts the dates chronologically
    equity_market_return = pd.concat(processed_frames, axis=1, sort=True)
    columns = [x for x in equity_market_return.columns if x.endswith('(USD)')]
    equity_market_return = equity_market_return[columns]
    
    print(f"Success. Joined {len(processed_frames)} assets.")
    print(f"Shape: {equity_market_return.shape}")
else:
    equity_market_return = pd.DataFrame()

print(f"Yields loaded: {yields_2yr is not None}")
print(f"Equity Markets loaded: {len(equity_market_return.columns)} ({list(equity_market_return.columns)})")

Success. Joined 7 assets.
Shape: (550, 7)
Yields loaded: True
Equity Markets loaded: 7 (['US___SPX_Index_Price (USD)', 'CH___MXCH_Index_Price (USD)', 'AU___MXAU_Index_Price (USD)', 'JP___MXJP_Index_Price (USD)', 'EU___MXEM_Index_Price (USD)', 'EM___MXEF_Index_Price (USD)', 'CH___SPI_Index_Price (USD)'])


/var/folders/p9/l40tvhkx4mgfss50nv3s08w80000gn/T/ipykernel_2908/4089677167.py:30: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return df.resample('M').last()
/var/folders/p9/l40tvhkx4mgfss50nv3s08w80000gn/T/ipykernel_2908/4089677167.py:30: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return df.resample('M').last()
/var/folders/p9/l40tvhkx4mgfss50nv3s08w80000gn/T/ipykernel_2908/4089677167.py:30: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return df.resample('M').last()
/var/folders/p9/l40tvhkx4mgfss50nv3s08w80000gn/T/ipykernel_2908/4089677167.py:30: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return df.resample('M').last()
/var/folders/p9/l40tvhkx4mgfss50nv3s08w80000gn/T/ipykernel_2908/4089677167.py:30: FutureWarning: 'M' is deprecated and will be removed in a futu

In [57]:
equity_market_return.keys()

Index(['US___SPX_Index_Price (USD)', 'CH___MXCH_Index_Price (USD)',
       'AU___MXAU_Index_Price (USD)', 'JP___MXJP_Index_Price (USD)',
       'EU___MXEM_Index_Price (USD)', 'EM___MXEF_Index_Price (USD)',
       'CH___SPI_Index_Price (USD)'],
      dtype='object')

In [58]:
df_changes = pd.DataFrame()

for country in equity_market_return.columns:
        print(country)
        print(equity_market_return[country].diff(12))
        df_changes[country] = equity_market_return[country].diff(12).fillna(0)

df_changes.head()

US___SPX_Index_Price (USD)
Date
1980-01-31          NaN
1980-02-29          NaN
1980-03-31          NaN
1980-04-30          NaN
1980-05-31          NaN
                ...    
2025-06-30    2564.1699
2025-07-31    2795.6818
2025-08-31    2784.0216
2025-09-30    3149.6626
2025-10-31    3807.5114
Freq: ME, Name: US___SPX_Index_Price (USD), Length: 550, dtype: float64
CH___MXCH_Index_Price (USD)
Date
1980-01-31         NaN
1980-02-29         NaN
1980-03-31         NaN
1980-04-30         NaN
1980-05-31         NaN
                ...   
2025-06-30    589.4947
2025-07-31    309.1648
2025-08-31    321.3094
2025-09-30    349.9577
2025-10-31    570.6135
Freq: ME, Name: CH___MXCH_Index_Price (USD), Length: 550, dtype: float64
AU___MXAU_Index_Price (USD)
Date
1980-01-31         NaN
1980-02-29         NaN
1980-03-31         NaN
1980-04-30         NaN
1980-05-31         NaN
                ...   
2025-06-30    695.9077
2025-07-31    579.9507
2025-08-31    601.1102
2025-09-30    238.0085
2025-10-31

,US___SPX_Index_Price (USD),CH___MXCH_Index_Price (USD),AU___MXAU_Index_Price (USD),JP___MXJP_Index_Price (USD),EU___MXEM_Index_Price (USD),EM___MXEF_Index_Price (USD),CH___SPI_Index_Price (USD)
Date,,,,,,,
1980-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980-02-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def calculate_pure_momentum_weights_yields(yields_df, tickers, lookback=12):
    """
    Method 2: Calculates pure cross-sectional momentum weights.
    NO volatility scaling or leverage applied.
    """
    # --- 1. Calculate Trends ---
    df_changes = pd.DataFrame()
    
    for country, ticker in tickers.items():
        if ticker in yields_df.columns:
            # Calculate 12-month change in yield
            df_changes[country] = yields_df[ticker].diff(lookback)

    # --- 2. Generate Raw Signals (Cross-Sectional) ---
    # Calculate Global Average of yield changes
    df_changes['Global_Avg_Change'] = df_changes.mean(axis=1)
    
    df_signals = pd.DataFrame()
    for country in tickers.keys():
        # Relative Trend: Asset Trend - Average Trend
        df_signals[country] = df_changes[country] - df_changes['Global_Avg_Change']

    # --- 3. Normalize to Base Weights ---
    n_assets = len(tickers)
    
    # We use shift(1) to avoid look-ahead bias (using T-1 signal for T trade)
    # We divide by N to split the exposure roughly equally across the basket
    base_weights = df_signals.shift(1) / n_assets
    
    # --- 4. Apply Asset Class Multipliers ---
    asset_multiplier = {"equity": -1.0, "bond": -1.0, "fx": 1.0, "interest_rate": -1.0}
    final_weights_dict = {}

    for asset_class, multiplier in asset_multiplier.items():
        # Apply specific direction
        final_weights_dict[asset_class] = base_weights.multiply(multiplier).dropna()
        
    return final_weights_dict


def calculate_pure_momentum_weights_equity_market_returns(equity_returns_df, lookback=12):
    """
    Method 2: Calculates pure cross-sectional momentum weights.
    NO volatility scaling or leverage applied.
    """
    # --- 1. Calculate Trends ---
    df_changes = pd.DataFrame()
    
    for country in equity_returns_df.columns:
        if country in equity_returns_df.columns:
            # Calculate 12-month change in yield
            df_changes[country] = equity_returns_df[country].diff(lookback).fillna(0)

    # --- 2. Generate Raw Signals (Cross-Sectional) ---
    # Calculate Global Average of yield changes
    df_changes['Global_Avg_Change'] = df_changes.mean(axis=1)
    
    df_signals = pd.DataFrame()
    for country in equity_returns_df.columns:
        # Relative Trend: Asset Trend - Average Trend
        df_signals[country] = df_changes[country] - df_changes['Global_Avg_Change']

    # --- 3. Normalize to Base Weights ---
    n_assets = len(equity_returns_df.columns)
    
    # We use shift(1) to avoid look-ahead bias (using T-1 signal for T trade)
    # We divide by N to split the exposure roughly equally across the basket
    base_weights = df_signals.shift(1) / n_assets
    
    # --- 4. Apply Asset Class Multipliers ---
    asset_multiplier = {"equity": 1.0, "bond": -1.0, "fx": 1.0, "interest_rate": -1.0}
    final_weights_dict = {}

    for asset_class, multiplier in asset_multiplier.items():
        # Apply specific direction
        final_weights_dict[asset_class] = base_weights.multiply(multiplier).dropna()
        
    return final_weights_dict

In [ ]:
weights_risk_sentiment = calculate_pure_momentum_weights_equity_market_returns(equity_market_return, tickers, lookback=12)
weights_monetary_policy = calculate_pure_momentum_weights_yields(yields_2yr, tickers, lookback=12)

print(sum(weights["equity"].loc['2004-02-29'].values))
print(weights.keys())

0.0
dict_keys(['equity', 'bond', 'fx', 'interest_rate'])


In [ ]:
def calculate_equity_returns(weights, equity_market_return):
    #TODO
    pass

def calculate_bond_returns(weights, bond_market_return):
    #TODO
    pass

def calculate_fx_returns(weights, fx_market_return):
    #TODO
    pass

def calculate_interest_rate_returns(weights, interest_rate_market_return):
    #TODO
    pass

1.0000000000981446e-06
